# Makemore: part5 WaveNet

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [5]:
for x,y in zip(Xtr[:20], Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> y
.......y --> u
......yu --> h
.....yuh --> e
....yuhe --> n
...yuhen --> g
..yuheng --> .
........ --> d
.......d --> i
......di --> o
.....dio --> n
....dion --> d
...diond --> r
..diondr --> e
.diondre --> .
........ --> x
.......x --> a
......xa --> v
.....xav --> i
....xavi --> e


In [23]:
# Near copy paste of the layers we have developed in Part 3

# -----------------------------------------------------------------------------------------------
class Linear:
  
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # note: kaiming init
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

# -----------------------------------------------------------------------------------------------
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      if x.ndim == 2:
        dim = 0
      elif x.ndim == 3:
        dim = (0,1)
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

# -----------------------------------------------------------------------------------------------
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class Embedding:
  
  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim))
    
  def __call__(self, IX):
    self.out = self.weight[IX]
    return self.out
  
  def parameters(self):
    return [self.weight]

# -----------------------------------------------------------------------------------------------

class Flatten:

    def __call__(self, x):
        self.out = x.view(x.shape[0], -1)
        return self.out
        
    def parameters(self):
        return []
      
class FlattenConsecutive:
  
  def __init__(self, n):
    self.n = n
    
  def __call__(self, x):
    B, T, C = x.shape
    x = x.view(B, T//self.n, C*self.n)
    if x.shape[1] == 1:
      x = x.squeeze(1)
    self.out = x
    return self.out
  
  def parameters(self):
    return []

# -----------------------------------------------------------------------------------------------
class Sequential:
  
  def __init__(self, layers):
    self.layers = layers
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out
  
  def parameters(self):
    # get parameters of all layers and stretch them out into one list
    return [p for layer in self.layers for p in layer.parameters()]

In [24]:
torch.manual_seed(42); # seed rng for reproducibility

In [44]:
n_embd =24 # the dimensionality of the character embedding vectors
n_hidden = 128 # the number of neurons in the hidden layer of the MLP
# C = torch.randn((vocab_size, n_embd))
model = Sequential([
    Embedding(vocab_size, n_embd),
    FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(), 
    Linear(n_hidden, vocab_size)
])

# parameter init
with torch.no_grad():
  model.layers[-1].weight *= 0.1 # last layer make less confident

parameters = model.parameters()

print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in model.parameters():
    p.requires_grad = True

76579


In [45]:
%time

# same optimization as last time
max_steps = 250000
batch_size = 32
lossi = []
losstrains = []
lossdevs = []
ud = []

for i in range(max_steps):
  
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
    # forward pass
    logits = model(Xb)
    loss = F.cross_entropy(logits, Yb) # loss function
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    if i < 150000:
        lr = 0.1
    else:
        lr = 0.01 # step learning rate decay
    for ix, p in enumerate(parameters):
        p.data += -lr * p.grad
    
    # track stats
    if i % 5000 == 0: # print every once in a while
        # print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')

        with torch.no_grad():
            logits_train = model(Xtr)
            losstrain = F.cross_entropy(logits_train, Ytr)
            logits_dev = model(Xdev)
            lossdev = F.cross_entropy(logits_dev, Ydev)
            losstrains.append(losstrain.item())
            lossdevs.append(lossdev.item())
            print(f'{i:7d}/{max_steps:7d}: train {losstrain.item():.4f} val.{lossdev.item():.4f}')
            
    lossi.append(loss.log10().item())

    with torch.no_grad():
        ud.append([(lr*p.grad.std() / p.data.std()).log10().item() for p in parameters])
        

    # if i >= 1000:
    #     break
    # break
    

CPU times: user 2 μs, sys: 2 μs, total: 4 μs
Wall time: 6.91 μs
      0/ 250000: train 3.2563 val.3.2555
   5000/ 250000: train 2.1666 val.2.1852
  10000/ 250000: train 2.1123 val.2.1454
  15000/ 250000: train 2.0654 val.2.1115
  20000/ 250000: train 2.0625 val.2.1189
  25000/ 250000: train 2.0295 val.2.0922
  30000/ 250000: train 2.0021 val.2.0698
  35000/ 250000: train 1.9867 val.2.0628
  40000/ 250000: train 1.9884 val.2.0741
  45000/ 250000: train 1.9663 val.2.0600
  50000/ 250000: train 1.9678 val.2.0658
  55000/ 250000: train 1.9448 val.2.0515
  60000/ 250000: train 1.9391 val.2.0502
  65000/ 250000: train 1.9275 val.2.0476
  70000/ 250000: train 1.9250 val.2.0429
  75000/ 250000: train 1.9246 val.2.0528
  80000/ 250000: train 1.9157 val.2.0513
  85000/ 250000: train 1.9135 val.2.0484
  90000/ 250000: train 1.8997 val.2.0439
  95000/ 250000: train 1.9117 val.2.0585
 100000/ 250000: train 1.8979 val.2.0420
 105000/ 250000: train 1.8880 val.2.0421
 110000/ 250000: train 1.8906 val.

In [39]:
plt.plot(torch.tensor(loss).view(-1, 1000).mean(dim=1))

/var/folders/8x/wpzm7f6s11b6xfkh92wr20yh0000gp/T/ipykernel_7425/328698954.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  plt.plot(torch.tensor(loss).view(-1, 1000).mean(dim=1))


RuntimeError: shape '[-1, 1000]' is invalid for input of size 1

In [46]:
for layer in model.layers:
    layer.training = False

In [47]:
# evaluate the loss
@torch.no_grad() # this decorator disables gradient tracking inside pytorch
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    logits = model(x)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 1.7612354755401611
val 2.000199794769287


In [ ]:
# Naively scale to 8 characters
train 1.9163438081741333
val 2.034247875213623

train 1.9367018938064575
val 2.02449369430542

## performance log
- original (3 character context + 200 hidden neurons, 12K params): train 2.058, val 2.105
- context: 3 -> 8 (22K params): train 1.916, val 2.034
- flat -> hierarchical (22K params): train 1.937, val 2.024
- fix bug in batchnorm: train 1.911, val 2.0201
- scale up the network: n_embd 24, n_hidden 128 (76K params): train 1.766, val 1.985


In [16]:
# sample from the model

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
        # ------------
        # forward pass:
        logit = model(torch.tensor([context]))
        # Sample
        probs = F.softmax(logit, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    
    print(''.join(itos[i] for i in out))

whaliosan.
dylina.
jenelia.
kailot.
benoderia.
elexten.
elizan.
aurlend.
zeyelle.
isabe.
sifrand.
zaryiak.
emmetric.
ennahanda.
jasit.
leiley.
klowyn.
aidaliel.
sharuson.
cobusius.
